
# 📊 ChatGPT Review Analysis
### Comprehensive Sentiment & Text Analysis using Python

---

## 🟦 1️⃣ Project Overview

This project analyzes customer reviews of ChatGPT to understand:

- Overall user sentiment (Positive, Neutral, Negative)
- Strength of opinions (Subjectivity)
- Frequently praised or criticized features
- Sentiment trends over time

The analysis converts raw textual feedback into structured business insights using Natural Language Processing (NLP).



## 🟦 2️⃣ Import Required Libraries

We import libraries for:
- Data manipulation (Pandas, NumPy)
- Visualization (Matplotlib, Seaborn)
- NLP (TextBlob)
- Text processing (Regex, Counter)


In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from textblob import TextBlob
from wordcloud import WordCloud
from collections import Counter
import re

sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (10,6)



## 🟦 3️⃣ Load the Dataset

The dataset contains:
- Review ID
- Review Text
- Rating (0–5 scale)
- Review Date


In [ ]:

df = pd.read_csv("chatgpt_reviews.csv")
df.head()



## 🟦 4️⃣ Data Inspection

We inspect the structure, data types, and missing values.


In [ ]:

df.info()
df.isnull().sum()



## 🟦 5️⃣ Data Cleaning & Preparation

Steps performed:
- Standardize column names
- Handle missing reviews
- Convert ratings to numeric
- Convert dates to datetime
- Create new features for analysis


In [ ]:

df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

df = df.dropna(subset=["review"])

df["rating"] = pd.to_numeric(df["rating"], errors="coerce")
df["review_date"] = pd.to_datetime(df["review_date"], errors="coerce")

df = df.dropna(subset=["review_date"])

df["month"] = df["review_date"].dt.to_period("M")
df["review_length"] = df["review"].apply(len)

df.head()



## 🟦 6️⃣ Exploratory Data Analysis (EDA)

We analyze rating distribution and review patterns.


In [ ]:

sns.countplot(x="rating", data=df)
plt.title("Rating Distribution")
plt.show()

print("Average Rating:", round(df["rating"].mean(), 2))



## 🟦 7️⃣ Sentiment Analysis

We calculate:
- Polarity (Positive/Negative strength)
- Subjectivity (Opinion intensity)

Using TextBlob NLP library.


In [ ]:

def get_sentiment(text):
    analysis = TextBlob(text)
    return analysis.sentiment.polarity, analysis.sentiment.subjectivity

df[["polarity", "subjectivity"]] = df["review"].apply(
    lambda x: pd.Series(get_sentiment(x))
)

def categorize_sentiment(score):
    if score > 0:
        return "Positive"
    elif score < 0:
        return "Negative"
    else:
        return "Neutral"

df["sentiment"] = df["polarity"].apply(categorize_sentiment)

df.head()


In [ ]:

sns.countplot(x="sentiment", data=df)
plt.title("Sentiment Distribution")
plt.show()



### Subjectivity Distribution

Higher subjectivity indicates stronger personal opinions.


In [ ]:

sns.histplot(df["subjectivity"], bins=30)
plt.title("Subjectivity Distribution")
plt.show()



## 🟦 8️⃣ Keyword Extraction

We extract frequently mentioned words from positive and negative reviews.


In [ ]:

def extract_keywords(text_series):
    text = " ".join(text_series).lower()
    words = re.findall(r'\b[a-z]{3,}\b', text)
    return Counter(words).most_common(15)

positive_reviews = df[df["sentiment"]=="Positive"]["review"]
negative_reviews = df[df["sentiment"]=="Negative"]["review"]

print("Top Positive Keywords:")
print(extract_keywords(positive_reviews))

print("\nTop Negative Keywords:")
print(extract_keywords(negative_reviews))



## 🟦 9️⃣ Word Cloud Visualization


In [ ]:

positive_text = " ".join(positive_reviews)
wordcloud_pos = WordCloud(width=800, height=400, background_color="white").generate(positive_text)

plt.imshow(wordcloud_pos)
plt.axis("off")
plt.title("Positive Reviews Word Cloud")
plt.show()



## 🟦 🔟 Time-Series Analysis

Analyze sentiment trends over time.


In [ ]:

monthly_sentiment = df.groupby(["month", "sentiment"]).size().unstack()

monthly_sentiment.plot()
plt.title("Monthly Sentiment Trend")
plt.xlabel("Month")
plt.ylabel("Review Count")
plt.show()



## 🟦 1️⃣1️⃣ Key Insights

- Majority of reviews are Positive.
- Strong subjectivity appears in Negative reviews.
- Helpfulness and speed are commonly praised.
- Accuracy issues are frequently criticized.
- Sentiment fluctuates over time.

## 🟦 1️⃣2️⃣ Conclusion

This project demonstrates how NLP and data analytics can transform unstructured customer feedback into actionable insights. Continuous monitoring of sentiment trends can significantly improve product strategy and customer satisfaction.
